In [1]:
%load_ext autoreload
%autoreload 2

### Search for a geographic reference product for the Stack 

In [2]:
import asf_search as asf

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
# results = asf.product_search('S1_181296_IW1_20220219T125501_VV_10AF-BURST')
reference = results[0]
reference

### Create an `asf_search.Stack` object from the reference scene `asf_search.Products.S1Product.S1Product` object

Stack accepts ASFSearchOptions, which you can use to limit its temporal range.

In [3]:

args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2022-03-02'}
)

stack = asf.Stack(reference, opts=args)
stack.full_stack

[Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E),
 Pair(S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C, S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E),
 Pair(S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252),
 Pair(S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105